# Carregamento dos dados e das bibliotecas

Nessa etapa foi realizado a importação das bibliotecas e dos dados já rotulados na etapa anterior tanto o realizado a partir do aprendizado semi-supervisionado quanto os rotulados manualmente
 (**gold labels**).
Devido a quantidade massiva de dados foi necessário o salvamento dos dados em multiplos arquivos parquet, e depois foi realizado a concatenação em um único *DataFrame*.
 

## Importação das bibliotecas

In [ ]:
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import recall_score, f1_score, matthews_corrcoef



## Carregamento dos Dados

In [ ]:
load_dotenv()

SEED = 1303

dfs = []
path_dados = os.getenv("PATH_DADOS")


for path in os.listdir(path_dados):
    if path.startswith('df_com_pred'):
        

        df = pd.read_parquet(f'{path_dados}/{path}')
        dfs.append(df)

dfs = pd.concat(dfs)

dfs.head()


## Colocar onde fica esses dados.
path_gold_labels = 'Dados_de_teste.parquet'
df_gold_labels = pd.read_parquet(f'{folder_path}/{path}')
y_gold_labels = np.load(f'{folder_path}/rotulos_teste.npy')



# Seleção de Colunas Necessárias e Processamento dos Dados

Nesta etapa, focamos na preparação dos dados para garantir que somente os casos em que o modelo semi-supervisionado apresentou uma predição com alta confiança sejam considerados. Isso é fundamental para evitar que predições incertas impactem negativamente nas análises e no treinamento dos modelos subsequentes.

## Passos Realizados

### Seleção das Colunas Relevantes
Selecionamos as colunas essenciais para o processo:
- **texto:** Contém o conteúdo textual que servirá de base para as análises.
- **pred_prob:** Armazena as probabilidades atribuídas pelo modelo semi-supervisionado para cada classe.

### Filtragem das Probabilidades
O objetivo aqui é garantir que utilizemos apenas os casos em que o modelo demonstrou uma confiança razoável. Para isso, criamos duas novas colunas:
- **prob_negativa:** Recebe a probabilidade da classe negativa somente se for maior que 0.6.
- **prob_positiva:** Recebe a probabilidade da classe positiva somente se for maior que 0.6.

Esse procedimento assegura que apenas os registros com predições robustas (confiança superior a 60%) sejam mantidos.

### Remoção de Dados com Baixa Confiança
Após a filtragem, removemos as linhas que não possuem nenhuma das probabilidades acima do limiar definido. Essa limpeza garante que os dados com predições incertas não sejam incluídos nas análises futuras.

### Definição da Classe
Por fim, determinamos a classe final para cada registro com base na maior probabilidade (usando `np.argmax`). Assim, cada instância é classificada conforme o valor predito com maior confiança.


In [ ]:
# Define as colunas necessárias para análise
colunas_necessarias = ['texto', 'pred_prob']
df_final = dfs[colunas_necessarias].copy()

# Cria colunas para as probabilidades filtrando por um limiar de 0.6
df_final['prob_negativa'] = df_final['pred_prob'].apply(lambda x: x[0] if x[0] > 0.6 else None)
df_final['prob_positiva'] = df_final['pred_prob'].apply(lambda x: x[1] if x[1] > 0.6 else None)

# Remove linhas que não possuem nenhuma das probabilidades acima do limiar
df_final = df_final.dropna(subset=['prob_negativa', 'prob_positiva'], how='all')

# Define a classe com base na maior probabilidade
df_final['classe'] = df_final['pred_prob'].apply(lambda x: np.argmax(x))

# Exibe a distribuição relativa das classes
df_final.classe.value_counts(normalize=True)

# Divisão dos Dados em Conjuntos de Treino e Teste

Nesta etapa, os dados são particionados em conjuntos de treino e teste para garantir uma avaliação robusta do modelo. Essa divisão é essencial para testar a capacidade de generalização do modelo, evitando overfitting e garantindo que os resultados reflitam a performance em dados não vistos.

## Processo de Divisão

1. **Definição das Variáveis:**  
   - **X:** Conjunto de dados de entrada que contém o texto.  
   - **y:** Conjunto de dados de saída que contém a classe associada.

2. **Divisão com Estratificação:**  
   Utilizamos a função `train_test_split` para separar os dados em conjuntos de treino e teste, garantindo que a distribuição das classes seja mantida. Reservamos 10% dos dados para o conjunto de teste, permitindo uma avaliação consistente do desempenho do modelo.

3. **Amostragem para Ajuste do TF-IDF:**  
   Devido a limitações computacionais, foi necessário utilizar uma amostra de 10% dos dados de treino para o ajuste do TF-IDF. Essa abordagem possibilita um processamento mais eficiente sem comprometer significativamente a representatividade dos dados.

In [ ]:

X = df_final['texto']
y = df_final['classe']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.1,random_state=SEED,stratify=y)
X_to_fitTFidf=X_train.sample(0.1)


In [ ]:
tfidf_vec_ssl = TfidfVectorizer(ngram_range=(1,3), strip_accents='unicode', lowercase=True, max_features=3000, min_df=10)

X_vec = tfidf_vec_ssl.fit_transform(X_train)
X_vec = X_vec.toarray()
y_vec = y_train

# Define a grade de hiperparâmetros
param_grid = {
    # Diferentes combinações para as duas camadas ocultas:
    'hidden_layer_sizes': [(50, 50), (100, 50), (100, 100), (150, 50)],
    # Funções de ativação: tanh e logistic (sigmoid)
    'activation': ['tanh', 'logistic'],
    # Solver pode ser ajustado também, aqui usamos o 'adam' como exemplo
    'solver': ['adam']
}

# Cria o classificador MLP
mlp = MLPClassifier(max_iter=500, random_state=42)

# Configura o GridSearchCV para buscar os melhores parâmetros
grid_search = GridSearchCV(estimator=mlp,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=-1,
                           verbose=2)

# Executa a busca nos dados de treinamento
grid_search.fit(X_train, y_train)

# Exibe os melhores parâmetros e a pontuação associada
print("Melhores parâmetros encontrados:")
print(grid_search.best_params_)
print("Melhor pontuação (CV): {:.4f}".format(grid_search.best_score_))

# Avalia o modelo no conjunto de teste
score = grid_search.score(X_test, y_test)


In [ ]:
best_params={'activation': 'tanh', 'hidden_layer_sizes': (150, 50), 'solver': 'adam'}

In [ ]:
best_model = MLPClassifier(max_iter=500, random_state=1303,**best_params)
best_model.fit(X_vec,y_vec)

In [ ]:


X_test_vec = tfidf_vec_ssl.transform(X_test)
y_pred = best_model.predict(X_test_vec)

recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
mcc = matthews_corrcoef(y_test, y_pred)

print("Metric Summary:")
print("-" * 20)
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"MCC: {mcc:.4f}")

# Avaliação Final do Modelo com Dados Rotulados Manualmente

Nesta etapa, foram empregados dados rotulados manualmente (Gold Labels) para realizar a validação final do modelo desenvolvido. A utilização desses rótulos, que contam com a curadoria humana, possibilita uma avaliação precisa e confiável do desempenho do classificador.

Os textos presentes no conjunto Gold Labels foram primeiramente transformados utilizando a vetorização TF-IDF, e o modelo treinado foi então utilizado para realizar as predições correspondentes. Em seguida, foram calculadas métricas fundamentais para mensurar a performance do modelo:

- **Recall:** Avalia a capacidade do modelo em identificar corretamente as instâncias positivas.
- **F1 Score:** Combina as medidas de precisão e recall, oferecendo uma visão equilibrada do desempenho.
- **MCC (Coeficiente de Correlação de Matthews):** Fornece uma análise robusta do desempenho, considerando a correlação entre as classes, mesmo em casos de desequilíbrio.

Esta abordagem permite validar a eficácia do modelo de forma rigorosa, fornecendo uma base sólida para a interpretação dos resultados e para o aprimoramento de futuras implementações.


In [ ]:

X_gold_labels_vec = tfidf_vec_ssl.transform(df_gold_labels['texto'])
y_pred_gold_labels = best_model.predict(X_gold_labels_vec)

recall_gold = recall_score(y_gold_labels, y_pred_gold_labels)
f1_gold = f1_score(y_gold_labels, y_pred_gold_labels)
mcc_gold = matthews_corrcoef(y_gold_labels, y_pred_gold_labels)

print("\nGold Labels Metric Summary:")
print("-" * 20)
print(f"Recall: {recall_gold:.4f}")
print(f"F1 Score: {f1_gold:.4f}")
print(f"MCC: {mcc_gold:.4f}")
